### Import Packages

In [1]:

from omegaconf import DictConfig,OmegaConf
conf = OmegaConf.load('config/config.yaml')
pic_=conf['config']['pic_']

### 01 Hyper Parameter Tuning for Catboost Model
this code is performed in both artificial (Syntetic) and non artificial data.

In [2]:
print('==============')
print('Run Hyperparameter Tuning Catboost')
%run -i '05_a_Catboost-Optuna-MLFlow [No Artificial].py'
%run -i '05_a_Catboost-Optuna-MLFlow [With Artificial].py'
print('DONE')
print('==============')

Run Hyperparameter Tuning Catboost


/home/jupyter-peta_admin/05_a_Catboost-Optuna-MLFlow [No Artificial].py:44: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(tracking_uri=tracking_uri_,nest_trials=True,
/home/jupyter-peta_admin/05_a_Catboost-Optuna-MLFlow [No Artificial].py:69: ExperimentalWarning: track_in_mlflow is experimental (supported from v2.9.0). The interface can change in the future.
  @mlflc.track_in_mlflow()
[I 2022-08-05 08:05:06,315] A new study created in RDB with name: Hyper Parameter Tuning
/opt/tljh/user/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[I 2022-08-05 08:08:44,593] Trial 0 finished with values: [0.7105179885808626, 0.8872673227740545, 0.9457875811482076, 0.6084322851752908] and parameters: {'iterations': 1000, 'learning_rate': 0.098299284327979, 'depth': 13, 'colsample_bylevel': 0.08

[I 2022-08-05 08:19:36,176] Trial 17 finished with values: [0.5970145146866617, 1.2732308766268008, 0.9243601666897687, 0.4827586472327403] and parameters: {'iterations': 100, 'learning_rate': 0.06425069111210799, 'depth': 6, 'colsample_bylevel': 0.07769710320013176, 'boosting_type': 'Plain', 'min_data_in_leaf': 199, 'l2_leaf_reg': 2.297628619695355e-06, 'auto_class_weights': 'Balanced', 'bootstrap_type': 'MVS'}. 
[I 2022-08-05 08:19:41,529] Trial 18 finished with values: [0.45613262708949576, 1.777866378828207, 0.902110254229252, 0.35198067658204046] and parameters: {'iterations': 100, 'learning_rate': 0.07227083926229155, 'depth': 12, 'colsample_bylevel': 0.029239870193410608, 'boosting_type': 'Plain', 'min_data_in_leaf': 213, 'l2_leaf_reg': 0.800935995600887, 'auto_class_weights': 'Balanced', 'bootstrap_type': 'MVS'}. 
[I 2022-08-05 08:20:14,735] Trial 19 finished with values: [0.7641466602462682, 0.7538348524097466, 0.9429040144479446, 0.6597035120793511] and parameters: {'iteratio

[I 2022-08-05 08:36:31,899] Trial 35 finished with values: [0.7117424502992364, 1.2198083926372234, 0.9095935284238541, 0.5199770618029984] and parameters: {'iterations': 100, 'learning_rate': 0.0761349543889819, 'depth': 7, 'colsample_bylevel': 0.012753498652356877, 'boosting_type': 'Plain', 'min_data_in_leaf': 13, 'l2_leaf_reg': 0.00010296501260210773, 'auto_class_weights': 'SqrtBalanced', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.8220689459054618}. 
[I 2022-08-05 08:37:01,185] Trial 36 finished with values: [0.6168673041205202, 1.5765298883810905, 0.9151585200820854, 0.4978754891167171] and parameters: {'iterations': 1000, 'learning_rate': 0.020195043115211603, 'depth': 12, 'colsample_bylevel': 0.01882335383683766, 'boosting_type': 'Plain', 'min_data_in_leaf': 37, 'l2_leaf_reg': 0.5203230764772001, 'auto_class_weights': 'Balanced', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 8.78108559554157}. 
[I 2022-08-05 08:38:02,506] Trial 37 finished with values: [0.78193

[I 2022-08-05 08:53:24,233] Trial 2 finished with values: [0.5304628632938644, 1.6283481019146269, 0.9000732490663909, 0.48155490768968195] and parameters: {'iterations': 100, 'learning_rate': 0.022393706289599743, 'depth': 10, 'colsample_bylevel': 0.0775924113963192, 'boosting_type': 'Plain', 'min_data_in_leaf': 161, 'l2_leaf_reg': 0.8345701321639168, 'auto_class_weights': 'SqrtBalanced', 'bootstrap_type': 'MVS'}. 
[I 2022-08-05 08:53:44,153] Trial 3 finished with values: [0.5548618586293506, 1.2420041379507059, 0.9049065081502239, 0.511801495765441] and parameters: {'iterations': 1000, 'learning_rate': 0.04583396294606061, 'depth': 13, 'colsample_bylevel': 0.029311201818206824, 'boosting_type': 'Plain', 'min_data_in_leaf': 143, 'l2_leaf_reg': 0.11589488064824043, 'auto_class_weights': 'Balanced', 'bootstrap_type': 'Bernoulli', 'subsample': 0.4403435484601125}. 
[I 2022-08-05 08:53:49,610] Trial 4 finished with values: [0.5483315392895587, 1.3601243812824504, 0.9049794509537419, 0.504

[I 2022-08-05 09:00:00,288] Trial 20 finished with values: [0.4670254754216002, 2.2631052461044905, 0.8878673306797223, 0.4084336335553994] and parameters: {'iterations': 100, 'learning_rate': 0.004705121982955792, 'depth': 12, 'colsample_bylevel': 0.061126551236144536, 'boosting_type': 'Plain', 'min_data_in_leaf': 43, 'l2_leaf_reg': 0.005548307822636225, 'auto_class_weights': 'SqrtBalanced', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 9.825275084597724}. 
[I 2022-08-05 09:00:05,606] Trial 21 finished with values: [0.5018299246501614, 1.5292180783363167, 0.8914055959460032, 0.4498633453251581] and parameters: {'iterations': 200, 'learning_rate': 0.06915233123504501, 'depth': 14, 'colsample_bylevel': 0.026252978083180945, 'boosting_type': 'Plain', 'min_data_in_leaf': 2, 'l2_leaf_reg': 0.001057164196253787, 'auto_class_weights': 'SqrtBalanced', 'bootstrap_type': 'Bernoulli', 'subsample': 0.21679495032410018}. 
[I 2022-08-05 09:00:09,355] Trial 22 finished with values: [0.4357373

[I 2022-08-05 09:04:07,124] Trial 38 finished with values: [0.4220308575529243, 2.2414627511074956, 0.8690172903155549, 0.3644091499948806] and parameters: {'iterations': 100, 'learning_rate': 0.01350861191344526, 'depth': 6, 'colsample_bylevel': 0.03364801557302886, 'boosting_type': 'Plain', 'min_data_in_leaf': 163, 'l2_leaf_reg': 0.6444203975308384, 'auto_class_weights': 'Balanced', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 9.99777804106931}. 
[I 2022-08-05 09:04:28,703] Trial 39 finished with values: [0.6200932902762827, 1.0588105133569055, 0.9213196724344105, 0.5823684441021875] and parameters: {'iterations': 500, 'learning_rate': 0.04522987648651457, 'depth': 10, 'colsample_bylevel': 0.09452948280452507, 'boosting_type': 'Plain', 'min_data_in_leaf': 159, 'l2_leaf_reg': 0.0002478158568048178, 'auto_class_weights': 'SqrtBalanced', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 3.3329182549048078}. 
[I 2022-08-05 09:04:51,283] Trial 40 finished with values: [0.564549

DONE


### 02 Hyper Parameter Tuning for LightGBM Model
this code is performed in both artificial (Syntetic) and non artificial data.

In [3]:
print('==============')
print('Run Hyperparameter Tuning Catboost')
%run -i '05_b_LightGBM-Optuna-MLFlow [No Artifcial].py'
%run -i '05_b_LightGBM-Optuna-MLFlow [With Artifcial].py'
print('DONE')
print('==============')

Run Hyperparameter Tuning Catboost


/home/jupyter-peta_admin/05_b_LightGBM-Optuna-MLFlow [No Artifcial].py:44: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(tracking_uri=tracking_uri_,nest_trials=True,
/home/jupyter-peta_admin/05_b_LightGBM-Optuna-MLFlow [No Artifcial].py:68: ExperimentalWarning: track_in_mlflow is experimental (supported from v2.9.0). The interface can change in the future.
  @mlflc.track_in_mlflow()
[I 2022-08-05 09:07:25,477] A new study created in RDB with name: Hyper Parameter Tuning
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/optuna/distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Pl

[LightGBM] [Warning] lambda_l1 is set=1.1921887171715278e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1921887171715278e-06
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] min_gain_to_split is set=7.540028488540534, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=7.540028488540534
[LightGBM] [Warning] lambda_l2 is set=1.1921887171715278e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.1921887171715278e-06
[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=2400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2400
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=1.1921887171715278e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.1921887171715278e-06
[LightGBM] [Warning] bagging_fraction is set=0.2, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2
[LightGBM] [Warning] min_gain_to_split is set=7.540028488540534, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=7.540028488540534
[LightGBM] [Warning] lambda_l2 is set=1.1921887171715278e-06, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.1921887171715278e-06
[LightGBM] [Warning] feature_fraction is set=0.30000000000000004, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.30000000000000004
[LightGBM] [Warning] min_data_in_leaf is set=2400, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=2400
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


...

[LightGBM] [Warning] lambda_l1 is set=0.6503313133947296, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6503313133947296
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_gain_to_split is set=1.4159722210274799, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.4159722210274799
[LightGBM] [Warning] lambda_l2 is set=0.6503313133947296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6503313133947296
[LightGBM] [Warning] feature_fraction is set=0.6000000000000001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6000000000000001
[LightGBM] [Warning] min_data_in_leaf is set=3800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3800
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l1 is set=0.6503313133947296, reg_alpha=0.0 wi

/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/jupyter-peta_admin/.local/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=0.6503313133947296, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6503313133947296
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_gain_to_split is set=1.4159722210274799, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=1.4159722210274799
[LightGBM] [Warning] lambda_l2 is set=0.6503313133947296, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6503313133947296
[LightGBM] [Warning] feature_fraction is set=0.6000000000000001, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6000000000000001
[LightGBM] [Warning] min_data_in_leaf is set=3800, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=3800
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


[I 2022-08-05 09:24:53,118] Trial 49 finished with values: [0.10620739146035163, 2.444269772860683, 0.5, 0.0] and parameters: {'iterations': 100, 'learning_rate': 0.0523189979557169, 'num_leaves': 1020, 'max_depth': 7, 'min_data_in_leaf': 3800, 'l2_leaf_reg': 0.6503313133947296, 'min_gain_to_split': 1.4159722210274799, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.6000000000000001}. 


DONE


### 03 Hyper Parameter Tuning for RandomForest Model
this code is performed in both artificial (Syntetic) and non artificial data.

In [4]:
print('==============')
print('Run Hyperparameter Tuning Catboost')
%run -i '05_c_RandomForest-Optuna-MLFlow [No Artifcial].py'
%run -i '05_c_RandomForest-Optuna-MLFlow [With Artifcial].py'
print('DONE')
print('==============')

Run Hyperparameter Tuning Catboost


/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:45: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(tracking_uri=tracking_uri_,nest_trials=True,
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:70: ExperimentalWarning: track_in_mlflow is experimental (supported from v2.9.0). The interface can change in the future.
  @mlflc.track_in_mlflow()
[I 2022-08-05 09:24:54,296] A new study created in RDB with name: Hyper Parameter Tuning
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A column-vector y wa

/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A

/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
[I 2022-08-05 09:26:25,301] Trial 8 finished with values: [0.5919378138543028, 1.4457996406673206, 0.9105329409209968, 0.4701967648369035] and parameters: {'iterations': 50, 'max_depth': 5, 'min_samples_split': 142, 'min_samples_leaf': 31, 'criterion': 'gini', 'max_leaf_nodes': 10, 'class_weight': 'balanced', 'bootstrap': True}. 
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial

/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [No Artifcial].py:92: DataConversionWarning: A

...

[I 2022-08-05 09:37:55,472] Trial 34 finished with values: [0.49278794402583426, 1.3968675094834573, 0.890680758820093, 0.44537688479271137] and parameters: {'iterations': 50, 'max_depth': 6, 'min_samples_split': 31, 'min_samples_leaf': 7, 'criterion': 'gini', 'max_leaf_nodes': 10, 'class_weight': 'balanced_subsample', 'bootstrap': True}. 
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Artifcial].py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Artifcial].py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow

/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Artifcial].py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Artifcial].py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Artifcial].py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
[I 2022-08-05 09:38:22,209] Trial 39 finished with values: [0.4659490491567994, 1.428127737085696

[I 2022-08-05 09:38:50,598] Trial 43 finished with values: [0.47965554359526374, 1.378924710158153, 0.8914684230803249, 0.4231412118286988] and parameters: {'iterations': 200, 'max_depth': 12, 'min_samples_split': 68, 'min_samples_leaf': 26, 'criterion': 'log_loss', 'max_leaf_nodes': 9, 'class_weight': None, 'bootstrap': False}. 
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Artifcial].py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Artifcial].py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Art

/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Artifcial].py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Artifcial].py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
/home/jupyter-peta_admin/05_c_RandomForest-Optuna-MLFlow [With Artifcial].py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_.fit(train_x.iloc[train_index,],train_y.iloc[train_index,])
[I 2022-08-05 09:39:16,740] Trial 48 finished with values: [0.3745245783997129, 1.656864402778604

DONE


### 04 Comparison Model and Bundle the Result into Pickle
this code is performed in both artificial (Syntetic) and non artificial data.

In [2]:
import mlflow
from mlflow.tracking import MlflowClient
model_tuning={}

In [4]:
client = MlflowClient()
experiment = client.get_experiment_by_name("Hyper Parameter Tuning")
if experiment!=None:
    exp_id=experiment.experiment_id
    data_=mlflow.search_runs([exp_id], order_by=["metrics.f1_micro DESC","metrics.logloss ASC","metrics.roc_auc_score DESC"])

#### Result the of hypertuning model

In [6]:
data_.groupby(['params.algorithm','params.strategy']).first()

run_id  \
params.algorithm params.strategy                                     
catboost         no artificial    f60203247a2c4e0087bcf2cd6fb46728   
                 with artificial  84545001b5c943738610d92a063946b8   
lightgbm         no artificial    47438690da5a47bc83cb33ce37d5d5db   
                 with artificial  667c8ab2b09244809eced9dbfeb9c751   
randomforest     no artificial    b2c703ed8109427696f3e7eca9858d7e   
                 with artificial  6828283bf2604a20a0e154d35c2991b9   

                                 experiment_id    status  \
params.algorithm params.strategy                           
catboost         no artificial               2  FINISHED   
                 with artificial             2  FINISHED   
lightgbm         no artificial               2  FINISHED   
                 with artificial             2  FINISHED   
randomforest     no artificial               2  FINISHED   
                 with artificial             2  FINISHED   

                                                                       artifact_uri  \
params.algorithm params.strategy                                                      
catboost         no artificial    ./mlruns/2/f60203247a2c4e0087bcf2cd6fb46728/ar...   
                 with artificial  ./mlruns/2/84545001b5c943738610d92a063946b8/ar...   
lightgbm         no artificial    ./mlruns/2/47438690da5a47bc83cb33ce37d5d5db/ar...   
                 with artificial  ./mlruns/2/667c8ab2b09244809eced9dbfeb9c751/ar...   
randomforest     no artificial    ./mlruns/2/b2c703ed8109427696f3e7eca9858d7e/ar...   
                 with artificial  ./mlruns/2/6828283bf2604a20a0e154d35c2991b9/ar...   

                                                       start_time  \
params.algorithm params.strategy                                    
catboost         no artificial   2022-08-05 08:42:25.476000+00:00   
                 with artificial 2022-08-05 09:05:05.195000+00:00   
lightgbm         no artificial   2022-08-05 09:12:28.946000+00:00   
                 with artificial 2022-08-05 09:22:02.618000+00:00   
randomforest     no artificial   2022-08-05 09:32:07.339000+00:00   
                 with artificial 2022-08-05 09:37:13.951000+00:00   

                                                         end_time  \
params.algorithm params.strategy                                    
catboost         no artificial   2022-08-05 08:46:39.304000+00:00   
                 with artificial 2022-08-05 09:05:27.679000+00:00   
lightgbm         no artificial   2022-08-05 09:12:37.607000+00:00   
                 with artificial 2022-08-05 09:22:30.997000+00:00   
randomforest     no artificial   2022-08-05 09:32:18.634000+00:00   
                 with artificial 2022-08-05 09:37:18.207000+00:00   

                                  metrics.roc_auc_score  metrics.log_loss  \
params.algorithm params.strategy                                            
catboost         no artificial                 0.953784          0.649188   
                 with artificial               0.923251          1.007035   
lightgbm         no artificial                 0.948871          0.636149   
                 with artificial               0.923256          0.986222   
randomforest     no artificial                 0.932225          0.773460   
                 with artificial               0.894269          1.360197   

                                  metrics.cohen_kappa  metrics.f1_micro  ...  \
params.algorithm params.strategy                                         ...   
catboost         no artificial               0.690260          0.784701  ...   
                 with artificial             0.603721          0.639612  ...   
lightgbm         no artificial               0.675209          0.783738  ...   
                 with artificial             0.594203          0.631432  ...   
randomforest     no artificial               0.602600          0.747403  ...   
                 with artificial      

In [7]:
best_cat_no_artificial=data_.loc[(data_['params.algorithm']=='catboost')&
                                 (data_['params.strategy']=='no artificial')].iloc[0,:]['run_id']
best_cat_no_artificial='runs:/'+best_cat_no_artificial+'/catboost_model'
best_cat_no_artificial=mlflow.catboost.load_model(best_cat_no_artificial)

best_cat_w_artificial=data_.loc[(data_['params.algorithm']=='catboost')&
                                 (data_['params.strategy']=='with artificial')].iloc[0,:]['run_id']
best_cat_w_artificial='runs:/'+best_cat_w_artificial+'/catboost_model'
best_cat_w_artificial=mlflow.catboost.load_model(best_cat_w_artificial)


best_lgb_no_artificial=data_.loc[(data_['params.algorithm']=='lightgbm')&
                                 (data_['params.strategy']=='no artificial')].iloc[0,:]['run_id']
best_lgb_no_artificial='runs:/'+best_lgb_no_artificial+'/lgbm_model'
best_lgb_no_artificial=mlflow.lightgbm.load_model(best_lgb_no_artificial)

best_lgb_w_artificial=data_.loc[(data_['params.algorithm']=='lightgbm')&
                                 (data_['params.strategy']=='with artificial')].iloc[0,:]['run_id']
best_lgb_w_artificial='runs:/'+best_lgb_w_artificial+'/lgbm_model'
best_lgb_w_artificial=mlflow.lightgbm.load_model(best_lgb_w_artificial)


best_rf_no_artificial=data_.loc[(data_['params.algorithm']=='randomforest')&
                                 (data_['params.strategy']=='no artificial')].iloc[0,:]['run_id']
best_rf_no_artificial='runs:/'+best_rf_no_artificial+'/rf_model'
best_rf_no_artificial=mlflow.sklearn.load_model(best_rf_no_artificial)

best_rf_w_artificial=data_.loc[(data_['params.algorithm']=='randomforest')&
                                 (data_['params.strategy']=='with artificial')].iloc[0,:]['run_id']
best_rf_w_artificial='runs:/'+best_rf_w_artificial+'/rf_model'
best_rf_w_artificial=mlflow.sklearn.load_model(best_rf_w_artificial)

In [8]:
model_tuning['catboost_wo_na']=best_cat_no_artificial
model_tuning['lightgbm_wo_na']=best_lgb_no_artificial
model_tuning['randomforest_wo_na']=best_rf_no_artificial

model_tuning['catboost_w_a']=best_cat_w_artificial
model_tuning['lightgbm_w_a']=best_lgb_w_artificial
model_tuning['randomforest_w_a']=best_rf_w_artificial

### 05 Export Data
this code is performed in both artificial (Syntetic) and non artificial data.

In [9]:
with open('ml_output/04_05_modeling/tuning/model_tuned.pickle', 'wb') as handle:
    pickle.dump(model_tuning, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [10]:
import os
from google.cloud import storage
client = storage.Client()
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'config/gcp_store.json'
bucket = client.get_bucket('bps-gcp-bucket')
file_name='MLST2023/model/tuning-model/'+pic_+'_tuning.pickle'
bucket.blob(file_name).upload_from_filename('ml_output/04_05_modeling/tuning/model_tuned.pickle')